In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from pynq import Overlay
base = BaseOverlay("base.bit")

In [2]:
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
frame_in_w = 640
frame_in_h = 480

In [3]:
import cv2
face_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')

eye_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_eye.xml')

In [4]:
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("Capture device is open: " + str(videoIn.isOpened()))
outframe = hdmi_out.newframe()
ret, frame_vga = videoIn.read()

Capture device is open: True


In [5]:
import time

iterations = 30
start = time.time()
for i in range (iterations):
    ret, frame_vga = videoIn.read()     
    outframe = hdmi_out.newframe()
    outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
    hdmi_out.writeframe(outframe)
end = time.time()
print("USB WEBCAM display frequency is " + str(iterations/(end-start))  ) 


USB WEBCAM display frequency is 32.0071712030614


In [6]:
process_this_frame = True
#當按下板子上Button 3 按鈕時，跳出迴圈
while (base.buttons[3].read()==0):
    ret, frame_vga = videoIn.read()
    if process_this_frame:
        np_frame = frame_vga
        #將獲取的即時影像轉化為灰色 
        gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
        #檢測視訊中的人臉，並用vector儲存人臉的座標、大小（用矩形表示） 
        faces = face_cascade.detectMultiScale(gray, 1.3,5)
        #臉部檢測 
        for (x,y,w,h) in faces:
            cv2.rectangle(np_frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = np_frame[y:y+h, x:x+w]
            #檢測視訊中臉部的眼睛，並用vector儲存眼睛的座標、大小（用矩形表示） 
            eyes = eye_cascade.detectMultiScale(roi_gray)
            ## 眼睛偵測:
            for (ex,ey,ew,eh) in eyes:
                cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)  
    outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
    hdmi_out.writeframe(outframe)
print('End of this demo ...')

End of this demo ...


In [7]:
iterations = 60
while True:
    start = time.time()
    for i in range (iterations):
        ret, frame_vga = videoIn.read()
        np_frame = frame_vga
        gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(np_frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = np_frame[y:y+h, x:x+w]
        outframe = hdmi_out.newframe()
        outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
        hdmi_out.writeframe(outframe)    
    end_time = time.time()
    break
print("face detect frequncy is " + str(iterations/(end_time-start))  )

face detect frequncy is 3.447238755595912


In [8]:
videoIn.release()
hdmi_out.stop()
del hdmi_out